In [17]:
import cv2
import numpy as np
import os

In [18]:
# --------------------------
# Preprocessing function
# --------------------------
def preprocess_patch(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 11, 2)
    kernel = np.ones((3,3), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    # Sobel edges
    sobel_x = cv2.Sobel(cleaned, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(cleaned, cv2.CV_64F, 0, 1, ksize=3)
    sobel_mag = cv2.magnitude(sobel_x, sobel_y)
    edges = cv2.normalize(sobel_mag, None, 0, 255, cv2.NORM_MINMAX).astype("uint8")

    return edges

# --------------------------
# Helper: split image into grid patches
# --------------------------
def split_into_patches(img, grid):
    # Splits `img` into `grid x grid` patches and returns [((r,c), patch), ...]
    h, w = img.shape[:2]
    ph = h // grid
    pw = w // grid
    patches = []
    for r in range(grid):
        for c in range(grid):
            y1 = r * ph
            y2 = (r + 1) * ph if r < grid - 1 else h
            x1 = c * pw
            x2 = (c + 1) * pw if c < grid - 1 else w
            patch = img[y1:y2, x1:x2].copy()
            patches.append(((r, c), patch))
    return patches

In [19]:
# --------------------------
# Split image into N x N patches
# --------------------------
def split_into_patches(img, N):
    h, w = img.shape[:2]
    ph, pw = h // N, w // N
    patches = []
    for i in range(N):
        for j in range(N):
            patch = img[i*ph:(i+1)*ph, j*pw:(j+1)*pw]
            patches.append(((i,j), patch))
    return patches

In [20]:

# --------------------------
# Full pipeline with saving patches
# --------------------------
def process_folder_save(input_folder, output_root, patch_grid):
    os.makedirs(output_root, exist_ok=True)
    dataset_info = {}

    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print("Cannot read:", img_path)
            continue

        print("Processing:", img_name)
        image_output_dir = os.path.join(output_root, img_name.split('.')[0])
        os.makedirs(image_output_dir, exist_ok=True)

        patches = split_into_patches(img, patch_grid)
        dataset_info[img_name] = []

        for idx, ((row, col), patch) in enumerate(patches):
            edges = preprocess_patch(patch)

            # Save raw and processed patch
            cv2.imwrite(os.path.join(image_output_dir, f"patch_{idx:02d}_raw.png"), patch)
            cv2.imwrite(os.path.join(image_output_dir, f"patch_{idx:02d}_processed.png"), edges)

            # Store processed patch in memory (no contours or Fourier descriptors)
            patch_info = {
                "patch_raw": patch,
                "patch_processed": edges,
            }
            dataset_info[img_name].append(patch_info)

    return dataset_info


In [21]:
input_folder = "D:/Image_Proccessing/Project/Gravity Falls/puzzle_4x4"
output_root = "phase1_output_4x4"
patch_grid = 4

dataset_info = process_folder_save(input_folder, output_root, patch_grid)

# Access first patch info in memory
first_image = list(dataset_info.keys())[0]
first_patch = dataset_info[first_image][0]
print("Raw patch shape:", first_patch["patch_raw"].shape)
print("Available fields:", list(first_patch.keys()))


Processing: 0.jpg
Processing: 1.jpg
Processing: 10.jpg
Processing: 100.jpg
Processing: 101.jpg
Processing: 102.jpg
Processing: 103.jpg
Processing: 104.jpg
Processing: 105.jpg
Processing: 106.jpg
Processing: 107.jpg
Processing: 108.jpg
Processing: 109.jpg
Processing: 11.jpg
Processing: 12.jpg
Processing: 13.jpg
Processing: 14.jpg
Processing: 15.jpg
Processing: 16.jpg
Processing: 17.jpg
Processing: 18.jpg
Processing: 19.jpg
Processing: 2.jpg
Processing: 20.jpg
Processing: 21.jpg
Processing: 22.jpg
Processing: 23.jpg
Processing: 24.jpg
Processing: 25.jpg
Processing: 26.jpg
Processing: 27.jpg
Processing: 28.jpg
Processing: 29.jpg
Processing: 3.jpg
Processing: 30.jpg
Processing: 31.jpg
Processing: 32.jpg
Processing: 33.jpg
Processing: 34.jpg
Processing: 35.jpg
Processing: 36.jpg
Processing: 37.jpg
Processing: 38.jpg
Processing: 39.jpg
Processing: 4.jpg
Processing: 40.jpg
Processing: 41.jpg
Processing: 42.jpg
Processing: 43.jpg
Processing: 44.jpg
Processing: 45.jpg
Processing: 46.jpg
Process

In [22]:
input_folder = "D:/Image_Proccessing/Project/Gravity Falls/puzzle_2x2"
output_root = "phase1_output_2x2"
patch_grid = 2

dataset_info = process_folder_save(input_folder, output_root, patch_grid)

# Access first patch info in memory
first_image = list(dataset_info.keys())[0]
first_patch = dataset_info[first_image][0]
print("Raw patch shape:", first_patch["patch_raw"].shape)
print("Available fields:", list(first_patch.keys()))


Processing: 0.jpg
Processing: 1.jpg
Processing: 10.jpg
Processing: 100.jpg
Processing: 101.jpg
Processing: 102.jpg
Processing: 103.jpg
Processing: 104.jpg
Processing: 105.jpg
Processing: 106.jpg
Processing: 107.jpg
Processing: 108.jpg
Processing: 109.jpg
Processing: 11.jpg
Processing: 12.jpg
Processing: 13.jpg
Processing: 14.jpg
Processing: 15.jpg
Processing: 16.jpg
Processing: 17.jpg
Processing: 18.jpg
Processing: 19.jpg
Processing: 2.jpg
Processing: 20.jpg
Processing: 21.jpg
Processing: 22.jpg
Processing: 23.jpg
Processing: 24.jpg
Processing: 25.jpg
Processing: 26.jpg
Processing: 27.jpg
Processing: 28.jpg
Processing: 29.jpg
Processing: 3.jpg
Processing: 30.jpg
Processing: 31.jpg
Processing: 32.jpg
Processing: 33.jpg
Processing: 34.jpg
Processing: 35.jpg
Processing: 36.jpg
Processing: 37.jpg
Processing: 38.jpg
Processing: 39.jpg
Processing: 4.jpg
Processing: 40.jpg
Processing: 41.jpg
Processing: 42.jpg
Processing: 43.jpg
Processing: 44.jpg
Processing: 45.jpg
Processing: 46.jpg
Process

In [23]:
input_folder = "D:/Image_Proccessing/Project/Gravity Falls/puzzle_8x8"
output_root = "phase1_output_8x8"
patch_grid = 8

dataset_info = process_folder_save(input_folder, output_root, patch_grid)

# Access first patch info in memory
first_image = list(dataset_info.keys())[0]
first_patch = dataset_info[first_image][0]
print("Raw patch shape:", first_patch["patch_raw"].shape)
print("Available fields:", list(first_patch.keys()))


Processing: 0.jpg
Processing: 1.jpg
Processing: 10.jpg
Processing: 100.jpg
Processing: 101.jpg
Processing: 102.jpg
Processing: 103.jpg
Processing: 104.jpg
Processing: 105.jpg
Processing: 106.jpg
Processing: 107.jpg
Processing: 108.jpg
Processing: 109.jpg
Processing: 11.jpg
Processing: 12.jpg
Processing: 13.jpg
Processing: 14.jpg
Processing: 15.jpg
Processing: 16.jpg
Processing: 17.jpg
Processing: 18.jpg
Processing: 19.jpg
Processing: 2.jpg
Processing: 20.jpg
Processing: 21.jpg
Processing: 22.jpg
Processing: 23.jpg
Processing: 24.jpg
Processing: 25.jpg
Processing: 26.jpg
Processing: 27.jpg
Processing: 28.jpg
Processing: 29.jpg
Processing: 3.jpg
Processing: 30.jpg
Processing: 31.jpg
Processing: 32.jpg
Processing: 33.jpg
Processing: 34.jpg
Processing: 35.jpg
Processing: 36.jpg
Processing: 37.jpg
Processing: 38.jpg
Processing: 39.jpg
Processing: 4.jpg
Processing: 40.jpg
Processing: 41.jpg
Processing: 42.jpg
Processing: 43.jpg
Processing: 44.jpg
Processing: 45.jpg
Processing: 46.jpg
Process